**notebook di servizio con i passaggi per la risoluzione della EDA**

# Import dataset

In [23]:
%%capture
# scaricamento dati della f1.
# preso il mugello siccome hanno giraato nel 2020
# anche loro ed è il tempio della motogp

import fastf1
import polars as pl
from pathlib import Path

Path(Path.cwd()).mkdir(parents=True, exist_ok=True)


# crea una cartella 'cache' per velocizzare le esecuzioni future)
fastf1.Cache.enable_cache("cache")

# Q are the qualyfin
session = fastf1.get_session(2020, "Tuscan", "Q")
session.load()


driver = "LEC"
lap = session.laps.pick_driver(driver).pick_fastest()
telemetry = lap.get_telemetry()

# inutile importaare colonna "DRS"
df = pl.DataFrame(
    telemetry[["Time", "Speed", "RPM", "nGear", "Throttle", "Brake", "X", "Y", "Z"]]
)

# scale dei dati con rpm maggiori per "simulare" di più le moto
df = df.with_columns(
    [
        (pl.col("Time").dt.total_milliseconds() / 1000).alias("Time_Sec"),
        (pl.col("RPM") * 1.1).alias("RPM"),
    ]
)

parquet_name = "telemetry_mugello_f1_2020.parquet"

output_dir = Path.cwd() / "data"
output_dir.mkdir(parents=True, exist_ok=True)

df.write_parquet(output_dir / "mugello_telemetry_2020_f1_LEC.parquet")


In [24]:
df.head(10)  # type: ignore

Time,Speed,RPM,nGear,Throttle,Brake,X,Y,Z,Time_Sec
duration[ns],f64,f64,i64,f64,bool,f64,f64,f64,f64
0ns,302.704166,11918.206671,8,100.0,false,-975.26265,-376.235906,3028.122794,0.0
71ms,303.0,11913.0,8,100.0,false,-949.153334,-322.811421,3028.7839,0.071
207ms,303.566667,11920.48,8,100.0,false,-898.0,-220.0,3030.0,0.207
311ms,304.0,11926.2,8,100.0,false,-857.872403,-140.957297,3030.885684,0.311
427ms,304.0,11976.176667,8,100.0,false,-813.0,-53.0,3032.0,0.427
551ms,304.0,12029.6,8,100.0,false,-766.075722,39.97442,3033.559652,0.551
647ms,304.4,12025.64,8,100.0,false,-730.0,112.0,3035.0,0.647
791ms,305.0,12019.7,8,100.0,false,-675.667539,221.127492,3037.504216,0.791
867ms,305.633333,12041.296667,8,100.0,false,-647.0,279.0,3039.0,0.867


## Preprocessing iniziale

In [25]:
# siccome ci sono 8 marce in f1 e la moto ne ha 6, procedo a portare a 6 qualsiasi cosa maggiore di 6
df = df.with_columns(
    [
        pl.col("nGear").clip(1,6).alias("nGear")
    ]
)
df.head(10)

Time,Speed,RPM,nGear,Throttle,Brake,X,Y,Z,Time_Sec
duration[ns],f64,f64,i64,f64,bool,f64,f64,f64,f64
0ns,302.704166,11918.206671,6,100.0,false,-975.26265,-376.235906,3028.122794,0.0
71ms,303.0,11913.0,6,100.0,false,-949.153334,-322.811421,3028.7839,0.071
207ms,303.566667,11920.48,6,100.0,false,-898.0,-220.0,3030.0,0.207
311ms,304.0,11926.2,6,100.0,false,-857.872403,-140.957297,3030.885684,0.311
427ms,304.0,11976.176667,6,100.0,false,-813.0,-53.0,3032.0,0.427
551ms,304.0,12029.6,6,100.0,false,-766.075722,39.97442,3033.559652,0.551
647ms,304.4,12025.64,6,100.0,false,-730.0,112.0,3035.0,0.647
791ms,305.0,12019.7,6,100.0,false,-675.667539,221.127492,3037.504216,0.791
867ms,305.633333,12041.296667,6,100.0,false,-647.0,279.0,3039.0,0.867


In [26]:
df.describe()

statistic,Time,Speed,RPM,nGear,Throttle,Brake,X,Y,Z,Time_Sec
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""663""",663.0,663.0,663.0,663.0,663.0,663.0,663.0,663.0,663.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""","""0:00:38.122484""",247.594596,11994.683904,5.644042,83.290719,0.107089,-1458.002975,-1818.180611,3120.922904,38.122484
"""std""",null,46.44246,701.854992,0.629096,31.415795,null,2647.329811,3614.449624,112.211405,22.068684
"""min""","""0:00:00""",141.0,9991.3,4.0,0.0,0.0,-5622.540581,-7746.080753,2898.901935,0.0
"""25%""","""0:00:19.111000""",211.85,11695.2,5.0,82.0,null,-3470.623969,-4830.0,3037.0,19.111
"""50%""","""0:00:38.167000""",263.0,12146.2,6.0,99.0,null,-2074.0,-2184.0,3121.602562,38.167
"""75%""","""0:00:57.247000""",283.716668,12549.491429,6.0,100.0,null,1190.0,774.0,3191.10649,57.247
"""max""","""0:01:16.270000""",312.0,13200.0,6.0,100.0,1.0,3330.0,5418.0,3330.085494,76.27


In [27]:
import plotly.express as px

fig = px.scatter(
    df,
    x="X",
    y="Y",
    color="Speed", # colora in base alla velocita
    title="Mugello",
    color_continuous_scale="Turbo",  # rosso: veloce, blu:lento)
    width=800,
    height=800,
    hover_data=["RPM", "nGear", "Time_Sec"] # sono dati extraa qundo si passa col mouse sopra
)

fig.update_xaxes(visible=False) # toglie assi dal plot
fig.update_yaxes(visible=False)

fig.update_layout(
    template="plotly_dark",           
    plot_bgcolor="rgba(0,0,0,0)",     
    paper_bgcolor="rgba(0,0,0,0)",
    yaxis=dict(
        scaleanchor="x", 
        scaleratio=1  # scaleratio=1 così forza 1 metro su X a essere uguale a 1 metro su Y.
    )
)

fig.show()

scorrendo il mouse sul grafico, si può notare come ci siano praticamente solo 5 e 6 marcia, cosa un po infattibile per una moto. Vado a verificare nel df

In [28]:
df["nGear"].value_counts()

nGear,count
i64,u32
6,482
5,126
4,55


come pensavo, forse meglio ridimensionare in maniera più intelligente, alemno per includere una terza marcia. Faccio un offset di 2 $$8-2 = 6:$$ n marce moto

In [29]:
df = pl.read_parquet("data/mugello_telemetry_2020_f1_LEC.parquet")
df.head(10)

Time,Speed,RPM,nGear,Throttle,Brake,X,Y,Z,Time_Sec
duration[ns],f64,f64,i64,f64,bool,f64,f64,f64,f64
0ns,302.704166,11918.206671,8,100.0,false,-975.26265,-376.235906,3028.122794,0.0
71ms,303.0,11913.0,8,100.0,false,-949.153334,-322.811421,3028.7839,0.071
207ms,303.566667,11920.48,8,100.0,false,-898.0,-220.0,3030.0,0.207
311ms,304.0,11926.2,8,100.0,false,-857.872403,-140.957297,3030.885684,0.311
427ms,304.0,11976.176667,8,100.0,false,-813.0,-53.0,3032.0,0.427
551ms,304.0,12029.6,8,100.0,false,-766.075722,39.97442,3033.559652,0.551
647ms,304.4,12025.64,8,100.0,false,-730.0,112.0,3035.0,0.647
791ms,305.0,12019.7,8,100.0,false,-675.667539,221.127492,3037.504216,0.791
867ms,305.633333,12041.296667,8,100.0,false,-647.0,279.0,3039.0,0.867


In [30]:
df = df.with_columns(
    (pl.col("nGear") - 2).clip(1, 6).alias("nGear")
)

In [31]:
df["nGear"].value_counts().sort("nGear")

nGear,count
i64,u32
2,55
3,126
4,115
5,253
6,114


decisamente meglio

per non ogni volta fare copia incolla del codice che visualizza il circuito e anche in ottica futura per usarlo in streamlit, creare la funzione che metterò in `src/utils`.

In [32]:
def plot_track_map(df, color_by="Speed", title="Track map"):
    fig = px.scatter(
        df,
        x="X",
        y="Y",
        color=color_by, # colora in base alla velocita
        title=title,
        color_continuous_scale="Turbo",  # rosso: veloce, blu:lento)
        width=800,
        height=800,
        hover_data=["RPM", "nGear", "Time_Sec"] # sono dati extraa qundo si passa col mouse sopra
    )
    
    fig.update_traces(marker=dict(size=3))

    fig.update_xaxes(visible=False) # toglie assi dal plot
    fig.update_yaxes(visible=False)

    fig.update_layout(
        template="plotly_dark",           
        plot_bgcolor="rgba(0,0,0,0)",     
        paper_bgcolor="rgba(0,0,0,0)",
        yaxis=dict(
            scaleanchor="x", 
            scaleratio=1  # scaleratio=1 così forza 1 metro su X a essere uguale a 1 metro su Y.
        )
    )

    return fig

In [33]:
from src.graphics.charts import plot_track_map as plt_trckmp

fig = plt_trckmp(df, color_by="Speed", title="Mugello")
fig.show()

# EDA

In [34]:
df.describe()

statistic,Time,Speed,RPM,nGear,Throttle,Brake,X,Y,Z,Time_Sec
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""663""",663.0,663.0,663.0,663.0,663.0,663.0,663.0,663.0,663.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""","""0:00:38.122484""",247.594596,11994.683904,4.369532,83.290719,0.107089,-1458.002975,-1818.180611,3120.922904,38.122484
"""std""",null,46.44246,701.854992,1.207019,31.415795,null,2647.329811,3614.449624,112.211405,22.068684
"""min""","""0:00:00""",141.0,9991.3,2.0,0.0,0.0,-5622.540581,-7746.080753,2898.901935,0.0
"""25%""","""0:00:19.111000""",211.85,11695.2,3.0,82.0,null,-3470.623969,-4830.0,3037.0,19.111
"""50%""","""0:00:38.167000""",263.0,12146.2,5.0,99.0,null,-2074.0,-2184.0,3121.602562,38.167
"""75%""","""0:00:57.247000""",283.716668,12549.491429,5.0,100.0,null,1190.0,774.0,3191.10649,57.247
"""max""","""0:01:16.270000""",312.0,13200.0,6.0,100.0,1.0,3330.0,5418.0,3330.085494,76.27


si possono notare da un'analisi preliminare e ad alto livello come rpm max siamo molto vicini al limite dei 14_000 citata nel regolamento Articolo C.8.1.2.

Sarà utile metterla in una constante come variabile d'ambinete quando farò il docker compose

a differenza di quanto scritto nel test di selezione, non è concesso usare una live telemetry B.10.3.3

## Grafici EDA

### Speed in relazione alla velocità

In [35]:
import plotly.express as px

fig = px.box(
    df, 
    x="nGear", 
    y="Speed", 
    color="nGear",
    title="",
    template="plotly_dark",
    points="outliers" 
)

fig.update_layout(
    xaxis_title="marcia inserita",
    yaxis_title="velocità (km/h)",
    showlegend=False
)

fig.show()

#### Analisi spaziatura cambio
Ho plottato la distribuzione della velocità per ogni marcia per vedere se la rapportatura ha senso dopo aver traslato le marce della F1 di (-)2.

**NOTE**
- I boxplot non si sovrappongono: vuol dire che ogni marcia ha il suo range di utilizzo ben definito, non ci sono buchi di coppia.
- I baffi si toccano appena: ottimo, significa che quando cambio marcia il motore è già nel regime giusto.
- Quei puntini in basso (outliers) sotto la 5a e la 6a sono normali: sono le staccate violente (tipo San Donato) dove la velocità crolla prima che il pilota riesca a scalare fisicamente la marcia. Ma verificherò con il successivo grafico

In [36]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_zoom = df.filter(pl.col("Time_Sec") < 15).to_pandas()

fig = make_subplots(
    rows=2, cols=1, 
    shared_xaxes=True, 
    vertical_spacing=0.05,
    subplot_titles=("Analisi Velocità marce inserite", "Marce inserite (nGear)")
)

# rgrafico velocità
fig.add_trace(go.Scatter(
    x=df_zoom["Time_Sec"], 
    y=df_zoom["Speed"],
    mode='lines+markers',
    name='Velocità',
    marker=dict(size=6, color=df_zoom["Speed"], colorscale="Turbo"), # Colorato per velocità
    line=dict(color='#555')
), row=1, col=1)


# grafico marce
fig.add_trace(go.Scatter(
    x=df_zoom["Time_Sec"], 
    y=df_zoom["nGear"],
    mode='lines+markers',
    name='Marcia',
    line=dict(shape='hv', color='#ffea00', width=3), # hv per avere "effetto quantizzato", a gradino
    marker=dict(size=8)
), row=2, col=1)

brake_zones = df_zoom[df_zoom["Brake"] == True]

# "zoom" sui promi 15 secondi (definiti prima)
if not brake_zones.empty:
    x0 = brake_zones["Time_Sec"].iloc[0]
    x1 = brake_zones["Time_Sec"].iloc[-1]
    
    fig.add_vrect(x0=x0, x1=x1, fillcolor="red", opacity=0.2, layer="below", line_width=0, annotation_text="frenata", annotation_position="top left")

fig.update_layout(
    title_text="Analisi ritardo scalata (curva San Donato)", 
    template="plotly_dark", 
    height=700,
    hovermode="x unified" # così si possonon vedere i dati di entrambi i grafici passando il mouse
)

fig.show()

#### Verifica outliers (zoom San Donato)
Volevo confermare che quei puntini bassi nel boxplot non fossero errori del sensore. Ho fatto uno zoom sui primi 15 secondi (rettilineo -> T1).

**NOTE:**
- L'area rossa è dove il pilota sta frenando.
- Nel grafico sopra la velocità crolla da 300 a 200 km/h molto in fretta.
- Nel grafico sotto, però, la linea gialla resta piatta sulla 6a marcia per quasi un secondo pieno *mentre* la velocità sta già scendendo.
- Questo conferma la teoria: la velocità scende prima che avvenga la scalata meccanica. I dati sono validi.

### RPM (se vicino al limite)

In [37]:
fig = px.histogram(
    df, 
    x="RPM", 
    nbins=100, 
    title="Engine RPM",
    template="plotly_dark",
    color_discrete_sequence=['#FFD700'], 
    opacity=0.8
)

#  il limite regolamentare (Art. C.8.1.2)
fig.add_vline(
    x=14000, 
    line_dash="dash", 
    line_color="red", 
    annotation_text="Rev Limit (14k)", 
    annotation_position="top left"
)

# la media (dal describe)
mean_rpm = df["RPM"].mean()
fig.add_vline(
    x=mean_rpm, 
    line_dash="dot", 
    line_color="cyan", 
    annotation_text=f"Avg RPM: {int(mean_rpm)}",
    annotation_position="bottom left"
)

fig.show()

#### Distribuzione utilizzo motore
Volevo verificare due cose: se rispettasse il regolamento e se si stesse sfruttando a pieno il motore.

**Note:**
- Il picco massimo è attorno ai 13.200 giri. Quidni ci si trova sotto il muro dei 14.000 (Art. C.8.1.2), nessun problema con il regolamento.
- Il grafico è tutto spostato a destra: dovrebbe voler dire che il pilota riesce a tenere il motore sempre "in coppia" (tra 11.5k e 13k) siccome i motori racing girano "alti". Se la rapportatura fosse sbagliata avremmo avuto una distribuzione più piatta o picchi a bassi giri.

### Heatmap correlation matrix

In [38]:
corr_matrix = df.select(
    ["Speed", "RPM", "nGear", "Throttle", "Brake"]
).corr()

fig = px.imshow(
    corr_matrix.to_numpy(),
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    text_auto=".2f", 
    title="Correlation matrix",
    template="plotly_dark",
    color_continuous_scale="RdBu_r", 
    aspect="auto"
)

fig.show()

#### Controllo coerenza dati
Ho usato la matrice di correlazione veloce per vedere se c'è qualche sensore che desse problemi/non funzionasse a modo.

**Note:**
- Speed vs nGear (0.93): altissima. Conferma che non ci sono errori di lettura sul sensore marce o slittamenti strani della frizione.
- Throttle vs Brake (-0.81): negativa come deve essere. Se uno sale l'altro scende. E non è -1 fisso perché ci sono momenti di transizione (freno motore o trail braking in ingresso curva) dove non sono perfettamente opposti.
- I dati sono fisicamente coerenti, posso mostrarli successivamente in app.

## Calcolo delle forze G 

In [39]:
import numpy as np
import plotly.graph_objects as go

# non avendo gli accelerometri, uso la cinematica:
# Longitudinale = derivata della velocità
# Laterale = Velocità * Velocità di rotazione (cambio di direzione)

df_physics = df.with_columns([
    (pl.col("Speed") / 3.6).alias("v_ms"), # km/h -> m/s
    pl.col("Time_Sec").diff().fill_null(0.2).alias("dt"), # Delta Tempo
    pl.col("X").diff().fill_null(0).alias("dx"),
    pl.col("Y").diff().fill_null(0).alias("dy")
])

# calcolo Long G (Accelerazione/Frenata)
# a = delta_v / delta_t / 9.81
df_physics = df_physics.with_columns(
    (pl.col("v_ms").diff().fill_null(0) / pl.col("dt") / 9.81).alias("Long_G_Raw")
)

dx = df_physics["dx"].to_numpy()
dy = df_physics["dy"].to_numpy()
heading = np.arctan2(dy, dx) 

heading_smooth = np.unwrap(heading)

df_physics = df_physics.with_columns(pl.Series("heading", heading_smooth))

df_physics = df_physics.with_columns(
    ((pl.col("v_ms") * pl.col("heading").diff().fill_null(0) / pl.col("dt")) / 9.81).alias("Lat_G_Raw")
)

df_physics = df_physics.with_columns([
    pl.col("Long_G_Raw").rolling_mean(window_size=8).fill_null(0).alias("Long_G"),
    pl.col("Lat_G_Raw").rolling_mean(window_size=8).fill_null(0).alias("Lat_G")
])

plot_data = df_physics.to_pandas()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=plot_data["Lat_G"],
    y=plot_data["Long_G"],
    mode='markers',
    marker=dict(
        size=4,
        color=plot_data["Speed"], 
        colorscale='Turbo',
        showscale=True,
        colorbar=dict(title="Speed (km/h)")
    ),
    text=plot_data["Time_Sec"], 
    hoverinfo='text+x+y',
    name='G-Force'
))

for r in [1.0, 2.0, 3.0, 4.0, 5.0]:
    fig.add_shape(type="circle",
        xref="x", yref="y",
        x0=-r, y0=-r, x1=r, y1=r,
        line_color="white", line_dash="dot", opacity=0.2
    )

fig.update_layout(
    title="G-G diagram",
    xaxis_title="Lateral G (Cornering)",
    yaxis_title="Longitudinal G (Braking/Accel)",
    template="plotly_dark",
    width=700, height=700,
    yaxis=dict(
        scaleanchor="x", 
        scaleratio=1,
        range=[-5.5, 5.5] # range F1 -> ovviamente le moto lo hanno inferiore (si odvrebbe attestare max 1.2, 1.5)
    ),
    xaxis=dict(range=[-5.5, 5.5]),
    annotations=[
        dict(x=0, y=5, text="ACCELERAZIONE", showarrow=False, font=dict(color="gray")),
        dict(x=0, y=-5, text="FRENATA (San Donato)", showarrow=False, font=dict(color="red")),
        dict(x=-5, y=0, text="CURVA SX", showarrow=False, font=dict(color="gray")),
        dict(x=5, y=0, text="CURVA DX", showarrow=False, font=dict(color="gray"))
    ]
)

fig.show()

In [40]:
import plotly.express as px

fig = px.density_heatmap(
    df, 
    x="RPM", 
    y="Throttle", 
    z="Speed",
    histfunc="count",
    title="Engine calibration map",
    template="plotly_dark",
    nbinsx=30, nbinsy=20,
    color_continuous_scale="Hot",
    
)

fig.update_layout(
    xaxis_title="Engine RPM",
    yaxis_title="Throttle (%)",
)

fig.show()

#### Engine calibration heatmap

**NOTE:**
1.  **WOT Dominance:** La concentrazione massima (bianco/giallo) è a `Throttle 100%`. Questo conferma la natura veloce del circuito (Mugello): il pilota passa la maggior parte del tempo a gas spalancato.
2.  **Mapping Focus:** Per la calibrazione della ECU (Mappa Benzina/Anticipo), non ha senso perdere tempo a ottimizzare le celle a metà gas (es. 40% throttle a 12k giri) perché sono zone scure (il motore non ci passa mai). Bisogna concentrare tutto lo sforzo di calibrazione sulla riga del 100%.
3.  **Parzializzato:** Le poche tracce a metà gas (zone rosso scuro) indicano la gestione dell'acceleratore in percorrenza delle curve lunghe (le 2 Arrabbiata/Bucine).